In [1]:
import pandas as pd

markets_file = "../data/markets.xlsx"


# Note: only take first 10 half-hour intervals while designing codebase
def read_market1_data(path: str) -> pd.DataFrame:
    return pd.read_excel(
        path,
        sheet_name="Half-hourly data",
        names=["Period_start", "Price"],
        index_col=0,
    ).iloc[0:10]


# Note: only take first 5 hour intervals while designing codebase
def read_market2_data(path: str) -> pd.DataFrame:
    market2 = pd.read_excel(
        markets_file,
        sheet_name="Hourly data",
        names=["Period_start", "Price"],
        index_col=0,
    ).iloc[0:5]

    # # Re-index to half-hourly periods, filling values forward from the hour
    # half_hours = pd.date_range(start=market2.index[0], end=market2.index[-1], freq='30min', name="Period_start")
    # return market2.reindex(half_hours, method='ffill')
    return market2


# Load market 1
market1_data = read_market1_data(markets_file)

# Load market 2
market2_data = read_market2_data(markets_file)

In [2]:
from batsim.core import Battery, Market, Operator, Timepoint
from batsim.io import write_commitments

market1 = Market("market1", price_interval=30)
market2 = Market("market2", price_interval=60)

battery = Battery()

operator = Operator()

n_timesteps = 10
# simulation_interval = 30  # mins
timepoints = [Timepoint() for _ in range(n_timesteps)]
commitments = []
for timepoint in timepoints:
    commitment = operator.determine_commitment(timepoint, battery, market1, market2)
    battery.execute(commitment)
    commitments.append(commitment)

write_commitments(commitments, "../data/simulation.csv")